<a href="https://colab.research.google.com/github/yekcel/AI-Agent-MRI-Image-Defacing/blob/main/Multi_agnet_deface_MRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q ultralytics opencv-python google-generativeai pillow numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.4 MB/s eta 0:00:00


In [18]:
import os
import cv2
import numpy as np
import google.generativeai as genai
from ultralytics import YOLO
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import json
from datetime import datetime


In [21]:
from google.colab import userdata

# Get API key from Colab secrets
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

# Initialize Gemini model
gemini_model = genai.GenerativeModel('gemini-2.0-flash-exp')

print("✅ Gemini API configured successfully!")


✅ Gemini API configured successfully!


In [22]:
# Create necessary directories
Path("input_images").mkdir(exist_ok=True)
Path("output_images").mkdir(exist_ok=True)
Path("logs").mkdir(exist_ok=True)

print("✅ Directories created:")
print("   • input_images/")
print("   • output_images/")
print("   • logs/")


✅ Directories created:
   • input_images/
   • output_images/
   • logs/


In [23]:
import requests

print("📥 Downloading YOLO face detection model...\n")

MODEL_URL = "https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/raw/main/model_outputs/best_model.pt"
MODEL_PATH = "yolo_face_detector.pt"

try:
    response = requests.get(MODEL_URL, timeout=60)
    response.raise_for_status()

    with open(MODEL_PATH, 'wb') as f:
        f.write(response.content)

    file_size = os.path.getsize(MODEL_PATH)
    print(f"✅ Model downloaded successfully!")
    print(f"   Size: {file_size:,} bytes")
    print(f"   Path: {MODEL_PATH}")

except Exception as e:
    print(f"❌ Error downloading model: {str(e)}")


📥 Downloading YOLO face detection model...

✅ Model downloaded successfully!
   Size: 5,471,386 bytes
   Path: yolo_face_detector.pt


In [24]:
class DetectorAgent:
    """
    Agent responsible for detecting faces in MRI images using YOLO.
    """

    def __init__(self, model_path: str = "yolo_face_detector.pt", confidence: float = 0.3):
        """
        Initialize the detector agent.

        Args:
            model_path: Path to YOLO model weights
            confidence: Detection confidence threshold
        """
        self.model_path = model_path
        self.confidence = confidence
        self.model = None

    def load_model(self):
        """Load the YOLO model."""
        if self.model is None:
            self.model = YOLO(self.model_path)
            print(f"✅ DetectorAgent: Loaded YOLO model from {self.model_path}")

    def detect(self, image: np.ndarray) -> List[Dict]:
        """
        Detect faces in the image.

        Args:
            image: Grayscale MRI image

        Returns:
            List of detection dictionaries with bbox and confidence
        """
        self.load_model()

        # Convert grayscale to RGB for YOLO
        if len(image.shape) == 2:
            image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        else:
            image_rgb = image

        # Run detection
        results = self.model(image_rgb, conf=self.confidence, verbose=False)

        detections = []
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            conf = box.conf[0].item()

            detections.append({
                'bbox': (x1, y1, x2, y2),
                'confidence': conf
            })

        return detections

print("✅ DetectorAgent class defined")


✅ DetectorAgent class defined


In [25]:
class DefacerAgent:
    """
    Agent responsible for applying non-reversible defacing strategies.
    """

    def __init__(self):
        """Initialize the defacer agent."""
        pass

    def deface(self, image: np.ndarray, bbox: Tuple[int, int, int, int],
               strategy: str = "gentle") -> np.ndarray:
        """
        Apply defacing strategy to the detected face region.

        Args:
            image: Original grayscale MRI image
            bbox: Bounding box (x1, y1, x2, y2)
            strategy: One of ['gentle', 'moderate', 'maximum', 'skull_strip']

        Returns:
            Defaced image
        """
        result_image = image.copy()
        x1, y1, x2, y2 = bbox

        # Ensure coordinates are within image bounds
        h, w = image.shape[:2]
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)

        if strategy == "gentle":
            # Inpainting using Telea algorithm
            mask = np.zeros(image.shape[:2], dtype=np.uint8)
            mask[y1:y2, x1:x2] = 255
            result_image = cv2.inpaint(result_image, mask, 3, cv2.INPAINT_TELEA)

        elif strategy == "moderate":
            # Random texture fill from surrounding pixels
            surrounding = []
            pad = 10
            if y1 > pad:
                surrounding.append(image[y1-pad:y1, x1:x2])
            if y2 < h - pad:
                surrounding.append(image[y2:y2+pad, x1:x2])
            if x1 > pad:
                surrounding.append(image[y1:y2, x1-pad:x1])
            if x2 < w - pad:
                surrounding.append(image[y1:y2, x2:x2+pad])

            if surrounding:
                texture = np.concatenate([s.flatten() for s in surrounding])
                random_fill = np.random.choice(texture, size=(y2-y1, x2-x1))
                result_image[y1:y2, x1:x2] = random_fill
            else:
                result_image[y1:y2, x1:x2] = 0

        elif strategy == "maximum":
            # Black box (complete removal)
            result_image[y1:y2, x1:x2] = 0

        elif strategy == "skull_strip":
            # Extended removal towards skull
            expansion = int((x2 - x1) * 0.3)
            y1_ext = max(0, y1 - expansion)
            y2_ext = min(h, y2 + expansion // 2)
            x1_ext = max(0, x1 - expansion // 2)
            x2_ext = min(w, x2 + expansion // 2)

            result_image[y1_ext:y2_ext, x1_ext:x2_ext] = 0

        return result_image

print("✅ DefacerAgent class defined")


✅ DefacerAgent class defined


In [26]:
class ValidatorAgent:
    """
    Agent responsible for validating defaced images (ensuring no faces remain).
    """

    def __init__(self, model_path: str = "yolo_face_detector.pt", confidence: float = 0.1):
        """
        Initialize the validator agent.

        Args:
            model_path: Path to YOLO model weights
            confidence: Lower confidence threshold for validation
        """
        self.model_path = model_path
        self.confidence = confidence
        self.model = None

    def load_model(self):
        """Load the YOLO model."""
        if self.model is None:
            self.model = YOLO(self.model_path)
            print(f"✅ ValidatorAgent: Loaded YOLO model")

    def validate(self, image: np.ndarray, threshold: float = 0.3) -> Dict:
        """
        Validate that no faces remain in the defaced image.

        Args:
            image: Defaced grayscale MRI image
            threshold: Maximum acceptable confidence for remaining faces

        Returns:
            Validation result dictionary
        """
        self.load_model()

        # Convert grayscale to RGB for YOLO
        if len(image.shape) == 2:
            image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        else:
            image_rgb = image

        # Run detection with lower confidence
        results = self.model(image_rgb, conf=self.confidence, verbose=False)

        detections = []
        max_confidence = 0.0

        for box in results[0].boxes:
            conf = box.conf[0].item()
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

            detections.append({
                'bbox': (x1, y1, x2, y2),
                'confidence': conf
            })

            max_confidence = max(max_confidence, conf)

        # Validation passes if no high-confidence faces remain
        is_valid = max_confidence < threshold

        return {
            'is_valid': is_valid,
            'detections': detections,
            'max_confidence': max_confidence,
            'num_faces': len(detections)
        }

print("✅ ValidatorAgent class defined")


✅ ValidatorAgent class defined


In [27]:
class GeminiStrategyAgent:
    """
    Agent that uses Gemini API to intelligently select defacing strategies.
    """

    def __init__(self, model):
        """
        Initialize the Gemini strategy agent.

        Args:
            model: Gemini generative model instance
        """
        self.model = model

    def decide_strategy(self, iteration: int, validation_result: Optional[Dict] = None,
                       previous_strategy: Optional[str] = None) -> str:
        """
        Use Gemini to decide the next defacing strategy.

        Args:
            iteration: Current iteration number (1-4)
            validation_result: Result from ValidatorAgent (None for first iteration)
            previous_strategy: Previously used strategy

        Returns:
            Strategy name: 'gentle', 'moderate', 'maximum', or 'skull_strip'
        """

        # Build prompt for Gemini
        if iteration == 1:
            prompt = """You are an AI agent helping to deface MRI images for privacy protection.
This is the FIRST attempt. Choose the best non-reversible defacing strategy from:
- gentle: Inpainting (subtle, tries to blend)
- moderate: Random texture fill (medium)
- maximum: Black box (complete removal)
- skull_strip: Extended removal towards skull

Choose ONE strategy. Respond with ONLY the strategy name."""

        else:
            max_conf = validation_result.get('max_confidence', 0.0)
            num_faces = validation_result.get('num_faces', 0)

            prompt = f"""You are an AI agent helping to deface MRI images for privacy protection.

ITERATION {iteration}/4
Previous strategy: {previous_strategy}
Validation result: {num_faces} faces detected, max confidence: {max_conf:.3f}

The previous strategy FAILED. Choose a MORE AGGRESSIVE non-reversible strategy from:
- gentle: Inpainting
- moderate: Random texture fill
- maximum: Black box
- skull_strip: Extended removal

IMPORTANT: Choose a strategy MORE AGGRESSIVE than '{previous_strategy}'.

Respond with ONLY the strategy name."""

        # Call Gemini
        response = self.model.generate_content(prompt)
        strategy = response.text.strip().lower()

        # Validate response
        valid_strategies = ['gentle', 'moderate', 'maximum', 'skull_strip']
        if strategy not in valid_strategies:
            # Fallback logic
            strategy_order = ['gentle', 'moderate', 'maximum', 'skull_strip']
            if previous_strategy and previous_strategy in strategy_order:
                idx = strategy_order.index(previous_strategy)
                strategy = strategy_order[min(idx + 1, len(strategy_order) - 1)]
            else:
                strategy = strategy_order[min(iteration - 1, len(strategy_order) - 1)]

        return strategy

print("✅ GeminiStrategyAgent class defined")


✅ GeminiStrategyAgent class defined


In [32]:
class MultiAgentOrchestrator:
    """
    Orchestrator that coordinates all agents in the defacing pipeline.
    """

    def __init__(self, max_iterations: int = 4, validation_threshold: float = 0.3):
        """
        Initialize the orchestrator.

        Args:
            max_iterations: Maximum number of defacing attempts
            validation_threshold: Confidence threshold for validation
        """
        self.detector = DetectorAgent()
        self.defacer = DefacerAgent()
        self.validator = ValidatorAgent()  # ✅ حذف threshold از اینجا
        self.strategy_agent = GeminiStrategyAgent(gemini_model)

        self.max_iterations = max_iterations
        self.validation_threshold = validation_threshold

    def run(self, image_path: str) -> Dict:
        """
        Run the complete multi-agent defacing pipeline.

        Args:
            image_path: Path to input MRI image

        Returns:
            Result dictionary with status, defaced image, and metadata
        """
        print(f"\n{'='*60}")
        print(f"🚀 Starting Multi-Agent Defacing Pipeline")
        print(f"{'='*60}\n")

        # Load image
        original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if original_image is None:
            return {'status': 'error', 'message': 'Failed to load image'}

        print(f"📸 Image loaded: {original_image.shape}")

        # Step 1: Initial Detection
        print(f"\n🔍 Step 1: Initial Face Detection")
        detections = self.detector.detect(original_image)

        if not detections:
            print("   ℹ️  No faces detected - image is already defaced/clean")
            return {
                'status': 'no_faces',
                'defaced_image': original_image,
                'iterations': 0,
                'strategies_used': []
            }

        print(f"   ✅ Detected {len(detections)} face(s)")
        for i, det in enumerate(detections, 1):
            print(f"      Face {i}: confidence={det['confidence']:.3f}, bbox={det['bbox']}")

        # Iterative defacing loop
        current_image = original_image.copy()
        strategies_used = []
        previous_strategy = None
        validation_result = None

        for iteration in range(1, self.max_iterations + 1):
            print(f"\n{'─'*60}")
            print(f"🔄 Iteration {iteration}/{self.max_iterations}")
            print(f"{'─'*60}")

            # Step 2: Gemini decides strategy
            print(f"\n🧠 Step 2: Gemini Strategy Selection")
            strategy = self.strategy_agent.decide_strategy(
                iteration=iteration,
                validation_result=validation_result,
                previous_strategy=previous_strategy
            )
            print(f"   📋 Selected strategy: '{strategy}'")
            strategies_used.append(strategy)

            # Step 3: Apply defacing
            print(f"\n✂️  Step 3: Applying Defacing")
            for det in detections:
                current_image = self.defacer.deface(
                    current_image,
                    det['bbox'],
                    strategy=strategy
                )
            print(f"   ✅ Defacing applied with '{strategy}' strategy")

            # Step 4: Validate result
            print(f"\n✔️  Step 4: Validation")
            validation_result = self.validator.validate(
                current_image,
                threshold=self.validation_threshold  # ✅ threshold رو اینجا پاس می‌دیم
            )

            print(f"   Faces detected: {validation_result['num_faces']}")
            print(f"   Max confidence: {validation_result['max_confidence']:.3f}")
            print(f"   Threshold: {self.validation_threshold}")

            if validation_result['is_valid']:
                print(f"   ✅ Validation PASSED!")
                print(f"\n{'='*60}")
                print(f"✅ SUCCESS - Image successfully defaced in {iteration} iteration(s)")
                print(f"{'='*60}")
                return {
                    'status': 'success',
                    'defaced_image': current_image,
                    'iterations': iteration,
                    'strategies_used': strategies_used,
                    'final_confidence': validation_result['max_confidence']
                }
            else:
                print(f"   ❌ Validation FAILED - trying next iteration")

            previous_strategy = strategy

        # Max iterations reached
        print(f"\n{'='*60}")
        print(f"⚠️  MAX ITERATIONS REACHED - Image may not be fully defaced")
        print(f"{'='*60}")
        return {
            'status': 'max_iterations',
            'defaced_image': current_image,
            'iterations': self.max_iterations,
            'strategies_used': strategies_used,
            'final_confidence': validation_result['max_confidence']
        }

print("✅ MultiAgentOrchestrator class defined")


✅ MultiAgentOrchestrator class defined


In [33]:
import requests
from pathlib import Path

print("📥 Downloading test MRI images from GitHub...")

# Create input folder
Path("input_images").mkdir(exist_ok=True)

# Correct RAW URLs for test images
TEST_IMAGE_URLS = [
    "https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/raw/main/test_data/images/dicom_000209_jpg.rf.68d7b19c0c5017332d18516147668c4f.jpg",
    "https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/raw/main/test_data/images/dicom_000218_jpg.rf.2d70d50536d38f2bfa4bd4699955db8b.jpg",
    "https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/raw/main/test_data/images/dicom_002284_jpg.rf.0ef17c3c1c6d85b222e00e3ec887c8ce.jpg",
]

success_count = 0

for idx, url in enumerate(TEST_IMAGE_URLS, 1):
    filename = url.split('/')[-1]
    print(f"   [{idx}/{len(TEST_IMAGE_URLS)}] Downloading {filename[:50]}...")

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        # Save file
        save_path = f"input_images/{filename}"
        with open(save_path, 'wb') as f:
            f.write(response.content)

        file_size = len(response.content)
        print(f"      ✅ Saved ({file_size:,} bytes)")
        success_count += 1

    except Exception as e:
        print(f"      ❌ Error: {str(e)}")

print(f"\n✅ Successfully downloaded {success_count}/{len(TEST_IMAGE_URLS)} images!")

# List downloaded images
files = os.listdir("input_images")
if files:
    print(f"\n📁 Files in input_images/:")
    for f in files:
        size = os.path.getsize(f"input_images/{f}")
        print(f"   • {f[:50]} ({size:,} bytes)")


📥 Downloading test MRI images from GitHub...
   [1/3] Downloading dicom_000209_jpg.rf.68d7b19c0c5017332d18516147668c...
      ✅ Saved (26,364 bytes)
   [2/3] Downloading dicom_000218_jpg.rf.2d70d50536d38f2bfa4bd4699955db...
      ✅ Saved (24,814 bytes)
   [3/3] Downloading dicom_002284_jpg.rf.0ef17c3c1c6d85b222e00e3ec887c8...
      ✅ Saved (21,033 bytes)

✅ Successfully downloaded 3/3 images!

📁 Files in input_images/:
   • dicom_000209_jpg.rf.68d7b19c0c5017332d18516147668c (26,364 bytes)
   • dicom_000218_jpg.rf.2d70d50536d38f2bfa4bd4699955db (24,814 bytes)
   • dicom_002284_jpg.rf.0ef17c3c1c6d85b222e00e3ec887c8 (21,033 bytes)


In [34]:
from ultralytics import YOLO
import cv2
import os

print("🧪 Testing YOLO model on downloaded images...\n")

model = YOLO("yolo_face_detector.pt")

# Get first image
images = [f for f in os.listdir("input_images") if f.endswith(('.jpg', '.png', '.jpeg'))]

if not images:
    print("❌ No images found in input_images/")
else:
    test_image_path = f"input_images/{images[0]}"
    print(f"Testing on: {images[0]}")

    # Load image as grayscale
    img_gray = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)

    if img_gray is None:
        print("❌ Failed to load image!")
    else:
        print(f"   ✅ Image loaded: {img_gray.shape} (grayscale)")

        # Convert grayscale to RGB (YOLO needs 3 channels)
        img_rgb = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)
        print(f"   🔄 Converted to RGB: {img_rgb.shape}")

        # Run detection
        results = model(img_rgb, conf=0.3, verbose=False)

        if len(results[0].boxes) > 0:
            print(f"   ✅ Detected {len(results[0].boxes)} face(s)")
            for i, box in enumerate(results[0].boxes, 1):
                conf = box.conf[0].item()
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                print(f"      Face {i}: confidence={conf:.3f}, bbox=({x1},{y1},{x2},{y2})")
        else:
            print("   ⚠️  No faces detected")
            print("      Try lowering confidence threshold")


🧪 Testing YOLO model on downloaded images...

Testing on: dicom_000209_jpg.rf.68d7b19c0c5017332d18516147668c4f.jpg
   ✅ Image loaded: (512, 512) (grayscale)
   🔄 Converted to RGB: (512, 512, 3)
   ✅ Detected 1 face(s)
      Face 1: confidence=0.689, bbox=(41,210,173,415)


In [35]:
# Initialize the orchestrator
orchestrator = MultiAgentOrchestrator(
    max_iterations=4,
    validation_threshold=0.3
)

print("\n✅ Orchestrator ready to process images!")
print(f"   • Max iterations: 4")
print(f"   • Validation threshold: 0.3 confidence")



✅ Orchestrator ready to process images!
   • Max iterations: 4
   • Validation threshold: 0.3 confidence


In [38]:
import os
from pathlib import Path

# Get list of input images
input_images = sorted([f for f in os.listdir("input_images")
                       if f.endswith(('.jpg', '.png', '.jpeg'))])

if not input_images:
    print("❌ No images found in input_images/ folder!")
else:
    # Process first image as test
    test_image = input_images[0]
    test_path = f"input_images/{test_image}"

    print(f"🎯 Processing test image: {test_image}\n")

    result = orchestrator.run(test_path)

    # Save result
    if 'defaced_image' in result:
        output_path = f"output_images/defaced_{test_image}"
        cv2.imwrite(output_path, result['defaced_image'])
        print(f"\n💾 Saved to: {output_path}")

    # Print summary
    print(f"\n{'='*60}")
    print("📊 PROCESSING SUMMARY")
    print(f"{'='*60}")
    print(f"Status: {result['status']}")
    print(f"Iterations: {result.get('iterations', 0)}")
    print(f"Strategies used: {' → '.join(result.get('strategies_used', []))}")
    if 'final_confidence' in result:
        print(f"Final confidence: {result['final_confidence']:.3f}")
    print(f"{'='*60}")


🎯 Processing test image: dicom_000209_jpg.rf.68d7b19c0c5017332d18516147668c4f.jpg


🚀 Starting Multi-Agent Defacing Pipeline

📸 Image loaded: (512, 512)

🔍 Step 1: Initial Face Detection
   ✅ Detected 1 face(s)
      Face 1: confidence=0.689, bbox=(41, 210, 173, 415)

────────────────────────────────────────────────────────────
🔄 Iteration 1/4
────────────────────────────────────────────────────────────

🧠 Step 2: Gemini Strategy Selection


TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash-exp
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-exp
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-exp
Please retry in 51.527728161s.